In [2]:
import sys, argparse, logging
import pymssql
import _mssql
import datetime
from decimal import Decimal
import pymongo
from pymongo import MongoClient
import re

from tmqr.settings import *

try:
    from tmqr.settings_local import *
except:
    pass


In [3]:
# Init mongo asset index
client = MongoClient(MONGO_CONNSTR)
mongo_db = client[MONGO_DB]
mongo_collection = mongo_db['asset_index']

In [3]:
mongo_collection.create_index([('contr', pymongo.ASCENDING), 
                               ('mkt', pymongo.ASCENDING), 
                               ('type', pymongo.ASCENDING)])

'contr_1_mkt_1_type_1'

In [4]:
mongo_collection.create_index([('instr', pymongo.ASCENDING), 
                               ('type', pymongo.ASCENDING),
                               ('exp', pymongo.ASCENDING)])

'instr_1_type_1_exp_1'

In [6]:
%%timeit
mongo_collection.find_one({'tckr': 'US.F.CLM83.830520'})

1000 loops, best of 3: 837 µs per loop


In [10]:
%%timeit
mongo_collection.find_one({'tckr': 'US.P.F-GCM18-180627.180425@860.0'})

1000 loops, best of 3: 800 µs per loop


In [11]:
len('US.P.F-GCM18-180627.180425@860.0')

32

In [9]:
%%timeit
regx = re.compile("^US.F.CLM83.*", re.IGNORECASE)
mongo_collection.find_one({'contr':  "GCM18", 'mkt': 'US', 'type': 'F'})
#mongo_collection.find_one({'tickr':  regx})

The slowest run took 8.53 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 833 µs per loop


In [40]:
%%timeit
mongo_collection.find({'instr': 'US.CL', 'mkt': 'US', 'type': 'P', 'exp': {'$gt': datetime.datetime(2015, 5, 20, 0, 0)}}).count()

100 loops, best of 3: 2.96 ms per loop


In [42]:
mongo_collection.explain().find({'instr': 'US.CL', 'mkt': 'US', 'type': 'P', 'exp': {'$gt': datetime.datetime(2015, 5, 20, 0, 0)}}).count()

TypeError: 'Collection' object is not callable. If you meant to call the 'explain' method on a 'Collection' object it is failing because no such method exists.

# Getting futures chains

In [10]:
%timeit list(mongo_collection.find({'type':'F', 'instr': 'US.CL'}).sort('exp', 1))

100 loops, best of 3: 7.71 ms per loop


In [7]:
[x['tckr'] for x in mongo_collection.find({'type':'F', 
                                           'instr': 'US.CL', 
                                           'exp': {'$gt': datetime.datetime(2011, 1, 1)}}, 
                                          projection=['tckr']).sort('exp', 1)]

['US.F.CL.G11.110120',
 'US.F.CL.H11.110222',
 'US.F.CL.J11.110322',
 'US.F.CL.K11.110419',
 'US.F.CL.M11.110520',
 'US.F.CL.N11.110622',
 'US.F.CL.Q11.110720',
 'US.F.CL.U11.110822',
 'US.F.CL.V11.110921',
 'US.F.CL.X11.111020',
 'US.F.CL.Z11.111121',
 'US.F.CL.F12.111221',
 'US.F.CL.G12.120120',
 'US.F.CL.H12.120222',
 'US.F.CL.J12.120321',
 'US.F.CL.K12.120420',
 'US.F.CL.M12.120522',
 'US.F.CL.N12.120620',
 'US.F.CL.Q12.120720',
 'US.F.CL.U12.120822',
 'US.F.CL.V12.120920',
 'US.F.CL.X12.121022',
 'US.F.CL.Z12.121120',
 'US.F.CL.F13.121220',
 'US.F.CL.G13.130122',
 'US.F.CL.H13.130220',
 'US.F.CL.J13.130320',
 'US.F.CL.K13.130422',
 'US.F.CL.M13.130522',
 'US.F.CL.N13.130620',
 'US.F.CL.Q13.130722',
 'US.F.CL.U13.130821',
 'US.F.CL.V13.130920',
 'US.F.CL.X13.131022',
 'US.F.CL.Z13.131120',
 'US.F.CL.F14.131220',
 'US.F.CL.G14.140122',
 'US.F.CL.H14.140220',
 'US.F.CL.J14.140320',
 'US.F.CL.K14.140422',
 'US.F.CL.M14.140521',
 'US.F.CL.N14.140620',
 'US.F.CL.Q14.140722',
 'US.F.CL.U